# IMEC2201 Herramientas Computacionales 
## Taller Semana 2: Visualizaciones y Ajuste de Datos
### Fecha Límite Entrega (Bloque Neón): <font color="#FF5733">23.59h de Abril 22, 2022</font>

Universidad de los Andes — Abril 6, 2022.

___
### Tener en Cuenta
La entrega del Taller 2 se puede realizar de tres maneras:
1. Cargar la solución en su repositorio en GitHub y enviar el enlace del mismo vía Bloque Neón.
2. Enviar vía Bloque Neón el archivo en Jupyter Notebook (extensión '.ipynb').
3. Enviar vía Bloque Neón el archivo PDF del Jupyter Notebook.
___

## Ejercicio 1: Correlación Medición-Simulación

En la carpeta $\texttt{data}$ se disponen los archivos `SD_5Min.xlsx` y `solar_production.csv`.

- `SD_5Min.xlsx` contiene las **mediciones** de los valores de producción de la planta solar fotovoltaica ubicada en el edificio Santo Domingo de la Universidad de los Andes, desde 1-Agosto-2019 hasta 30-Junio-2021.
- `solar_production.csv` contiene los resultados de una **simulación** de la misma planta y con la misma resolución de los datos (i.e., un dato cada cinco minutos), desde 1-Enero-2020 hasta 1-Enero-2021.

<div class="alert alert-block alert-warning">
    
Realice una correlación entre los datos medidos (eje X) y los datos simulados (eje Y) de Potencia AC. Tenga en cuenta:
1. ¿Cómo uniría los dos DataFrame en un único DataFrame con base en las fechas indicadas en el archivo `solar_production.csv`? Explore la función `innerjoin()`.
2. Cómo realizaría un ajuste lineal de los datos para: <br>
    2.1. Extraer parámetros de ecuación de la curva de regresión lineal (pendiente e intercepto). <br>
    2.2. Calcular el coeficiente de correlación R2. <br>
3. Grafique la correlación entre estos datos y su ajuste lineal, incluyendo en la leyenda la ecuación de la recta y el coeficiente de correlación.

**Nota:** Es posible experimentar retardos (lags) en VS Code o Jupyter Notebook por la cantidad de datos con que se trabaja y la alta resolución de las gráficas. Para evitar esto:
- Tomar el número de datos que ustedes desean graficar, ya sea modificando el DataFrame desde Julia o modificar el archivo CSV y guardarlo como un archivo aparte del original.
- Cambiar el backend de gráficas de Julia. Por ejemplo, para los que conocen matplotlib en Python, desde Julia pueden crear estas gráficas con la librería PyPlot.

</div>

___

**Ejemplo** <br>
<font color="#FF5733">Output</font>

<img src='./img/Correlation_Pac.png' width='350'/>

___

In [2]:
import Pkg
Pkg.activate()
Pkg.instantiate()

using Pkg
Pkg.add("XLSX")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Dates")

  Activating project at `C:\Users\djara\.julia\environments\v1.7`
    Updating registry at `C:\Users\djara\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\djara\.julia\environments\v1.7\Manifest.toml`


In [159]:


using DataFrames, XLSX, CSV, Dates

#Mediciones de SD_5Min

min_df = DataFrame(XLSX.readtable("./data/SD_5Min.xlsx", "Potencia_AC", header=true)...)
min_cop = copy(min_df)

for i in names(min_cop)
    min_cop[:, i] = replace(min_cop[:, i], missing => 0)
end

min_año = []
min_mes = []
min_dia = []
min_hor = []
min_mit = []

for i in min_cop[:,"Date"]
    push!(min_año, Dates.year(i))
    push!(min_mes, Dates.month(i))
    push!(min_dia, Dates.day(i))
    push!(min_hor, Dates.hour(Dates.DateTime(i)))
    push!(min_mit, Dates.minute(Dates.DateTime(i)))
end

min_cop[:, "Año"] = min_año 
min_cop[:, "Mes"] = min_mes 
min_cop[:, "Dia"] = min_dia
min_cop[:, "Hora"] = min_hor
min_cop[:, "Minuto"] = min_mit

min_df2 = min_cop[(min_cop[:, "Año"] .== 2020) .| (min_cop[:, "Año"] .== 2021) .& (min_cop[:, "Mes"] .== 1) .& (min_cop[:, "Dia"] .== 1) .& (min_cop[:, "Hora"] .== 0) .& (min_cop[:, "Minuto"] .== 0), :]

println(last(min_df2, 5))

In [23]:

#Simulacion de solar_production

sol_df = DataFrame(CSV.File("./data/solar_production.csv", header = 1,))
sol_cop = copy(sol_df)

for i in names(sol_cop)
    sol_cop[:, i] = replace(sol_cop[:, i], missing => 0)
end

sol_año = []
sol_mes = []
sol_dia = []
sol_hor = []
sol_mit = []

for i in sol_cop[:,"Datetime"]
    push!(sol_año, parse(Int64, i[1:4]))
    push!(sol_mes, parse(Int64, i[6:7]))
    push!(sol_dia, parse(Int64, i[9:10]))
    push!(sol_hor, parse(Int64, i[12:13]))
    push!(sol_mit, parse(Int64, i[15:16]))
end

sol_cop[:, "Año"] = sol_año 
sol_cop[:, "Mes"] = sol_mes 
sol_cop[:, "Dia"] = sol_dia
sol_cop[:, "Hora"] = sol_hor
sol_cop[:, "Minuto"] = sol_mit

pr = unique(sol_hor)
println(pr)


Any[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [179]:

#df = min_df2[:, ["AC Power SysB (W)", "AC Power SysA (W)"]]
#df[:, "AC Power Total"] = sum.(eachrow(df))

suma = []
suma2= []
sum  = []
sum_fi=[]

for i in min_df2[:, "AC Power SysB (W)"]
    push!(suma, i)
end

for i in  min_df2[:, "AC Power SysA (W)"]
    push!(suma2, i)
end

Float64.suma

#Float64.suma 
#Float64.suma2

#for i in length(suma)
#    sum = suma(i)+ suma2(i)
#    push!(sum_fi, sum)
#end
#sum_fi



ErrorException: type DataType has no field suma

## Ejercicio 2: Comportamiento Estadístico Mensual

En la carpeta $\texttt{data}$ se dispone el archivo `SD_5Min.xlsx` que contiene las **mediciones** de los valores de producción de la planta solar fotovoltaica ubicada en el edificio Santo Domingo de la Universidad de los Andes, desde 1-Agosto-2019 hasta 30-Junio-2021.

<div class="alert alert-block alert-warning">
    
Escriba un algoritmo que permita obtener los valores máximo, promedio y mínimo para todos los días de un mes durante cada instante de tiempo. Es decir:

<img src='./img/table_excercise2.png' width='650'/>

Note que:
- La primera columna contiene las estampas de tiempo para un día completo, es decir, un dato cinco minutal desde las 00:00 hasta las 23:55.
- Se tiene una columna de datos de irradiancia por día; es decir, 30 o 31 columnas de datos.
- Se agregan las columnas de interés: mínimo, promedio y máximo. Estas son columnas calculadas con las 30 o 31 columnas de datos. Por ejemplo:
    - Columna: Mínimo, Fila: 11:55 -> min(800, 700, 880, ..., 630) = 630
    - Columna: Promedio, Fila: 12:00 -> mean(850, 680, 930, ..., 720) = 795
    - Columna: Máximo, Fila: 12:05 -> max(1010, 810, 850, ..., 760) = 1010
- La gráfica se genera al graficar en una misma figura: (i.) Tiempo vs Mín, (ii.) Tiempo vs Prom, (iii.) Tiempo vs. Max.
</div>

___

**Ejemplo** <br>
<font color="#FF5733">Output</font>
<img src='./img/irradiance_per_minute.png' width='750'/>

**Nota:** Solo se solicita la gráfica de los datos mínimo, promedio y máximo, es decir, los marcadores color azul, verde y naranja en esta figura.
___

## Ejercicio 3: Histogramas y Distribuciones

En la carpeta $\texttt{data}$ se dispone el archivo `SD_5Min.xlsx` que contiene las **mediciones** de los valores de producción de la planta solar fotovoltaica ubicada en el edificio Santo Domingo de la Universidad de los Andes, desde 1-Agosto-2019 hasta 30-Junio-2021. 

<div class="alert alert-block alert-warning">
    
Similar al Ejercicio 2, escriba un algoritmo que permita obtener un DataFrame tal que:
- La primera columna contiene las estampas de tiempo para un día completo, es decir, un dato cinco minutal desde las 00:00 hasta las 23:55.
- Se tiene una columna de datos de irradiancia por día; es decir, 30 o 31 columnas de datos.

A partir de este DataFrame, cree un histogramas para tres instantes de tiempo diferentes (es decir, tres histogramas en total) y analice e interprete el resultado.

**Nota:** Explore qué es un *QQ-Plot* y cómo implementarlo en Julia.

</div>

___
**Ejemplo** <br>
<font color="#FF5733">Output</font>

<img src='./img/histograms.png' width='700'/>
___